# Reverse engineer molecules

#### Apr 2024 version

## Enumerate signatures and molecules from Morgan fingerprints

In [ ]:
# CHANGE TO YOUR DIRECTORY PATH
path_directory = "C:/Users/meyerp/Documents/INRAE/Diophantine/Enumération/github/signature"

# packages
import numpy as np
import os

import pandas as pd
import sys
import time

np.set_printoptions(threshold=sys.maxsize)

os.chdir(path_directory)

from rdkit.Chem.Descriptors import ExactMolWt
from rdkit import Chem
from rdkit import RDLogger

RDLogger.DisableLog("rdApp.*")

# Reverse engineer molecules from Morgan
from signature.enumerate_signature import enumerate_molecule_from_signature, enumerate_signature_from_morgan
from signature.enumerate_utils import reduced_fingerprint, test_sol_ECFP, test_sol_ECFP_reduced, test_sol_sig
from signature.Signature import MoleculeSignature
from signature.signature_alphabet import (
    load_alphabet,
    morgan_vector_from_signature,
    SignatureAlphabet,
    signature_from_smiles,
)
from signature.utils import read_csv

In [ ]:
# paths
file_smiles = "./datasets/MetaNetX_weight_500_radius_3_size_1000"
#file_smiles = "./datasets/emolecule_weight_500_radius_2_size_1000"
#file_smiles = "./datasets/emolecule_weight_500"
#file_smiles = "./datasets/dataset.test.small"

#file_alphabet = "./datasets/MetaNetX_weight_500_Alphabet_radius_2_nBits_2048_smarts"
#file_alphabet = "./datasets/MetaNetX_weight_500_Alphabet_radius_3_nBits_2048" # radius 3 smiles
#file_alphabet = "./datasets/MetaNetX_weight_500_r2_smiles_boundary_Alphabet_radius_2_nBits_2048"
#file_alphabet = "./datasets/MetaNetX_weight_500_r2_smiles_full_Alphabet_radius_2_nBits_2048"
#file_alphabet = "./datasets/MetaNetX_weight_500_Alphabet_radius_3_nBits_2048" # smiles r3
#file_alphabet = "./datasets/MetaNetX_weight_500_Alphabet_radius_2_boundary_nBits_2048" # smiles r2 boundary
file_alphabet = "./datasets/MetaNetX_weight_500_Alphabet_radius_2_nBits_2048_smarts_full" # smarts r2

# Load Smiles file
H, D = read_csv(file_smiles)
print(H, D.shape[0])
Smiles = np.asarray(list(set(D[:, 0])))
print(f"Number of smiles: {len(Smiles)}")

# Get alphabet
Alphabet = load_alphabet(file_alphabet)
Alphabet.print_out()

In [ ]:
Alphabet.use_smarts = True
Alphabet.boundary_bonds = False

- ECFP reduced => Signature

In [ ]:
list_i = []
list_smi = []
list_nsig = []
list_foundsig = []
list_ct = []
list_ct_solve = []
list_timeout = []

In [ ]:
Alphabet.nBits = 2048
max_nbr_partition = int(1e5)

for i in range(5):
    smi = D[i, 0]
    print(i, "|", smi)

    mol = Chem.MolFromSmiles(smi)
    ms = MoleculeSignature(mol, radius=Alphabet.radius, neighbor=True, use_smarts=Alphabet.use_smarts, nbits=False, boundary_bonds=Alphabet.boundary_bonds, map_root=True)
    sig = ms.as_deprecated_string(morgan=False, root=False, neighbors=True)
    morgan = reduced_fingerprint(smi, radius=Alphabet.radius, useFeatures=False)
    print(sig)
    st = time.time()

    Ssig, bool_timeout, ct_solve = enumerate_signature_from_morgan(
        morgan, Alphabet, max_nbr_partition=max_nbr_partition, method="partitions", verbose=False
    )
    foundsig = sig in Ssig
    ft = time.time() - st

    print(f"{i} | {smi} | {len(Ssig)} | {int(foundsig)} | {ft:.4f} | {ct_solve:.4f}| {bool_timeout}")

    list_i.append(i)
    list_smi.append(smi)
    list_nsig.append(len(Ssig))
    list_foundsig.append(foundsig)
    list_ct.append(ft)
    list_ct_solve.append(ct_solve)
    list_timeout.append(bool_timeout)

In [ ]:
100 * sum(list_foundsig) / len(list_foundsig)

In [ ]:
df = pd.DataFrame(
    {
        "ID": list_i,
        "smi": list_smi,
        "Nsig": list_nsig,
        "FoundSig": list_foundsig,
        "CT enum ecfp_red to sig": list_ct,
        "CT solve by partitions": list_ct_solve,
        "Partitions timeout": list_timeout,
    }
)

df

# df.to_excel("RevSig_ecfp_to_sig", index=False)

- Signature => Smiles

In [ ]:
list_i = []
list_smi = []
list_wt = []
list_nmol = []
list_foundmol = []
list_ct_mol = []
list_recursion_timeout = []

In [ ]:
Alphabet.nBits = 0
max_nbr_recursion = 1e5
max_nbr_solution = 1001
repeat = 10

print(f"ID | smi | weigth | Nmol | FoundMol | CPU-time | Timeout")
for i in range(5):
    smi = D[i, 0]
    print(i, "|", smi)
    wt = ExactMolWt(Chem.MolFromSmiles(smi))
    
    mol = Chem.MolFromSmiles(smi)
    ms = MoleculeSignature(mol, radius=Alphabet.radius, neighbor=True, use_smarts=Alphabet.use_smarts, nbits=False, boundary_bonds=Alphabet.boundary_bonds, map_root=True, legacy=False)
    sig = ms.as_deprecated_string(morgan=False, root=False, neighbors=True)

    st = time.time()
    smol, recursion_timeout = enumerate_molecule_from_signature(
        sig,
        Alphabet,
        smi,
        use_smarts=Alphabet.use_smarts,
        boundary_bonds=Alphabet.boundary_bonds,
        max_nbr_recursion=max_nbr_recursion,
        max_nbr_solution=max_nbr_solution,
        repeat=repeat,
        verbose=False,
    )

    ft = time.time() - st
    foundmol = smi in smol
    if foundmol == False:
        print(smol)

    print(f"{i} | {smi} | {wt:.1f} | {len(smol)} | {int(foundmol)} | {ft:.4f} | {recursion_timeout}")

    list_i.append(i)
    list_smi.append(smi)
    list_wt.append(wt)
    list_nmol.append(len(smol))
    list_foundmol.append(int(foundmol))
    list_ct_mol.append(ft)
    list_recursion_timeout.append(recursion_timeout)

In [ ]:
100 * sum(list_foundmol) / len(list_foundmol)

In [ ]:
df = pd.DataFrame(
    {
        "ID": list_i,
        "smi": list_smi,
        "weight": list_wt,
        "Nmol": list_nmol,
        "FoundMol": list_foundmol,
        "CT enum sig to mol": list_ct_mol,
        "Recursion timeout": list_recursion_timeout,
    }
)

df

# df.to_excel("RevSig_sig_to_mol", index=False)

- ECFP reduced => Signature => Smiles

In [ ]:
list_i = []
list_smi = []
list_wt = []
list_nsig = []
list_nsigtrue = []
list_foundsig = []
list_nmol = []
list_foundmol = []
list_ct_sig = []
list_ct_mol = []
list_ct_all = []
list_ct_solve = []

In [ ]:
max_nbr_partition = int(1e5)
max_nbr_recursion = 1e5
max_nbr_solution = 1001
repeat = 10

print(f"ID | smi | weigth | Nsig | NsigTrue | FoundSig | Nmol | FoundMol | CPU-time")
for i in range(5):
    # preparation
    smi = D[i, 0]
    print(i, "|", smi)
    wt = ExactMolWt(Chem.MolFromSmiles(smi))

    mol = Chem.MolFromSmiles(smi)
    ms = MoleculeSignature(mol, radius=Alphabet.radius, neighbor=True, use_smarts=Alphabet.use_smarts, nbits=False, boundary_bonds=Alphabet.boundary_bonds, map_root=True, legacy=False)
    sig = ms.as_deprecated_string(morgan=False, root=False, neighbors=True)
    morgan = reduced_fingerprint(smi, radius=Alphabet.radius, useFeatures=False)

    # ecfp_red => signature(s)
    st_1 = time.time()
    Alphabet.nBits = 2048
    Ssig, bool_timeout, ct_solve = enumerate_signature_from_morgan(
        morgan, Alphabet, max_nbr_partition=max_nbr_partition, method="partitions", verbose=False
    )
    foundsig = sig in Ssig
    ct_sig = time.time() - st_1
    print(f"...enumeratesignature: nbr signatures {len(Ssig)} CPU-time {ct_sig:.4f}")

    # signature(s) => molecule(s)
    st_2 = time.time()
    Smol, Nsig = set(), 0
    Alphabet.nBits = 0
    for j in range(len(Ssig)):
        smol, recursion_timeout = enumerate_molecule_from_signature(
            Ssig[j],
            Alphabet,
            smi,
            use_smarts=Alphabet.use_smarts,
            boundary_bonds=Alphabet.boundary_bonds,
            max_nbr_recursion=max_nbr_recursion,
            max_nbr_solution=max_nbr_solution,
            repeat=repeat,
            verbose=False,
        )
        if len(smol):
            Nsig += 1
            print(f"...enumeratemolecule:  signature {j} nbr molecules {len(smol)}")
        Smol = Smol | set(smol)
    ct_mol = time.time() - st_2

    ct_total = time.time() - st_1
    
    Alphabet.nBits = 2048
    Smolfinal = []
    for smi2 in Smol:
        if test_sol_ECFP_reduced([smi, smi2], Alphabet=Alphabet):
            Smolfinal.append(smi2)
    if len(Smol) != len(Smolfinal):
        print("Smol", len(Smol), "Smolfinal", len(Smolfinal))

    foundmol = smi in Smolfinal
    if foundmol == False:
        print(Smolfinal)
        
    print(
        f"{i} | {smi} | {wt:.1f} | {len(Ssig)} | {Nsig} | {int(foundsig)} | {len(Smolfinal)} | {int(foundmol)} | {ct_total:.4f} | {ct_solve:.4f}"
    )
    
    list_i.append(i)
    list_smi.append(smi)
    list_wt.append(wt)
    list_nsig.append(len(Ssig))
    list_nsigtrue.append(Nsig)
    list_foundsig.append(int(foundsig))
    list_nmol.append(len(Smolfinal))
    list_foundmol.append(int(foundmol))
    list_ct_sig.append(ct_sig)
    list_ct_mol.append(ct_mol)
    list_ct_all.append(ct_total)
    list_ct_solve.append(ct_solve)

In [ ]:
df = pd.DataFrame(
    {
        "ID": list_i,
        "smi": list_smi,
        "wt": list_wt,
        "Nsig": list_nsig,
        "NsigTrue": list_nsigtrue,
        "FoundSig": list_foundsig,
        "Nmol": list_nmol,
        "Foundmol": list_foundmol,
        "CT ecfp_sig": list_ct_sig,
        "CT sig_mol": list_ct_mol,
        "CT ecfp_mol": list_ct_all,
        "CT solve_partitions": list_ct_solve,
    }
)

df

# df.to_excel("RevSig_ecfp_to_mol_smiles_r2_boundary_500pe.xlsx", index=False)